In [0]:
import numpy as np
import torch
seed = 86
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(seed)

In [0]:
class ChebModel(torch.nn.Module):
    def __init__(self, m, n):
        super(ChebModel, self).__init__()
        self.A = torch.rand((m, n), dtype=torch.float32)
        self.b = torch.rand((m), dtype=torch.float32)
        self.x = torch.nn.Parameter(torch.rand((n)))
    
    def forward(self):
        return self.A @ self.x, self.b, self.x


# implementation of loss function for 11.21 in Boyd. Used simple barrier for
# l <= x <= u.
class ChebLoss(torch.nn.Module):
    def __init__(self):
        super(ChebLoss, self).__init__()
    
    def penalty(self, x, l, u):
        buf = 0
        if all(x < l) and all(x > u):
            buf = 1e8
        return buf

    def forward(self, ax, x, b, l, u):
        buf_loss = torch.max(torch.abs(torch.log(ax) - torch.log(b)))
        loss = buf_loss + self.penalty(x, l, u)
        return loss

In [36]:
# train loop
m = 5
n = 4
x0 = torch.rand((n))
l = torch.rand((n)) - 0.5 * torch.ones((n))
u = torch.rand((n)) + 0.5 * torch.ones((n))
model = ChebModel(m, n)
criterion = ChebLoss()
opt = torch.optim.Adam(model.parameters(), lr=1e-1)
epochs = 20
x = x0
for e in range(epochs):
    opt.zero_grad()
    ax, b, x = model()
    loss = criterion(ax, x, b, l, u)
    loss.backward()
    opt.step()
    print(loss.item())

1.4599155187606812
1.2929039001464844
1.0920997858047485
0.8404083251953125
1.158708930015564
1.1534702777862549
0.9957976937294006
0.7982392311096191
0.7310773730278015
0.791340172290802
0.801403820514679
0.7731611132621765
0.709769070148468
0.6084035634994507
0.7596017122268677
0.8102293014526367
0.6938321590423584
0.6101372241973877
0.6676223278045654
0.6879820227622986
